In [1]:
!git clone https://github.com/hanakim120/NLU_BERT.git

Cloning into 'NLU_BERT'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), 10.50 MiB | 3.21 MiB/s, done.


In [1]:
!pip install numpy==1.23

In [2]:
# install
!pip install datasets transformers sentencepiece accelerate -U evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00


In [4]:
import transformers
transformers.logging.set_verbosity_error()

In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
BASE_DIR = '/content/NLU_BERT/text-cls-nsmc'
os.chdir(BASE_DIR)
OUTPUT_DIR = os.path.join(BASE_DIR, 'checkpoint')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [9]:
import torch
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

- 훈련시키기

In [ ]:
!python kobert_ft_hftrainer.py --model_fn ./checkpoint/best_model.pth --train_fn ./data/nsmc_train.tsv --n_epoch 3

In [ ]:
def define_argparser():
    # command line argument를 받기 위한 함수
    p = argparse.ArgumentParser()

    p.add_argument('--model_fn', required=True) # 저장될 모델 파일 이름
    p.add_argument('--train_fn', required=True) # 학습에 사용될 파일 이름
    p.add_argument('--pretrained_model_name', type=str, default='beomi/kcbert-base') # 사전 학습된 모델 이름
    #  pretrained_model_name 예시:
    # - beomi/kcbert-base
    # - beomi/kcbert-large
    p.add_argument('--valid_ratio', type=float, default=.2) # valid set 비율
    p.add_argument('--batch_size_per_device', type=int, default=32) # device 당 batch size
    p.add_argument('--n_epochs', type=int, default=5) # epoch 수
    p.add_argument('--warmup_ratio', type=float, default=.2) # warmup 비율
    p.add_argument('--max_length', type=int, default=100) # 최대 길이

    config = p.parse_args() # argument를 config에 저장

    return config

In [ ]:
# 패키지 설치
import argparse # command line argument를 받기 위한 package
import random # random seed를 고정하기 위한 package
from sklearn.metrics import accuracy_score # 정확도 계산을 위한 package
import torch # pytorch package
from transformers import BertTokenizerFast # tokenizer package
from transformers import BertForSequenceClassification, AlbertForSequenceClassification # model package
from transformers import Trainer  # training을 위한 package
from transformers import TrainingArguments # training argument를 설정하기 위한 package
from transformers import DataCollatorWithPadding # padding을 위한 package

from utils.utils import read_text # 데이터 읽어오는 함수
from utils.bert_dataset import TextClassificationDataset # dataset class

In [ ]:
def define_argparser():
    # command line argument를 받기 위한 함수
    p = argparse.ArgumentParser()

    p.add_argument('--model_fn', required=True) # 저장될 모델 파일 이름
    p.add_argument('--train_fn', required=True) # 학습에 사용될 파일 이름
    p.add_argument('--pretrained_model_name', type=str, default='beomi/kcbert-base') # 사전 학습된 모델 이름
    #  pretrained_model_name 예시:
    # - beomi/kcbert-base
    # - beomi/kcbert-large
    p.add_argument('--valid_ratio', type=float, default=.2) # valid set 비율
    p.add_argument('--batch_size_per_device', type=int, default=32) # device 당 batch size
    p.add_argument('--n_epochs', type=int, default=5) # epoch 수
    p.add_argument('--warmup_ratio', type=float, default=.2) # warmup 비율
    p.add_argument('--max_length', type=int, default=100) # 최대 길이

    config = p.parse_args() # argument를 config에 저장

    return config

In [ ]:
def get_datasets(fn, tokenizer, valid_ratio=.2):
     # 데이터 파일을 읽어서 labels와 texts list 받기
    labels, texts = read_text(fn)

    # label을 index로 바꾸기 위한 과정
    unique_labels = list(set(labels)) # 중복 제거
    label_to_index = {} # label을 index로 바꾸기 위한 dict
    index_to_label = {} # index를 label로 바꾸기 위한 dict

    for i, label in enumerate(unique_labels):
        label_to_index[label] = i
        index_to_label[i] = label

    # labels 리스트의 각 요소를 label_to_index 딕셔너리에서 해당하는 값으로 변환하여 새로운 리스트를 생성
    labels = list(map(label_to_index.get, labels))

    # texts와 labels를 zip으로 묶고, random.shuffle로 섞기
    shuffled = list(zip(texts, labels))
    random.shuffle(shuffled)

    # 섞인 데이터를 다시 풀어서 texts와 labels로 나누기
    texts = [e[0] for e in shuffled]
    labels = [e[1] for e in shuffled]

    # valid set의 비율에 따라서 index 설정
    idx = int(len(texts) * (1 - valid_ratio))

    # train set과 valid set으로 나누기
    train_dataset = TextClassificationDataset(texts[:idx], labels[:idx], tokenizer) # 앞에서부터 idx까지
    valid_dataset = TextClassificationDataset(texts[idx:], labels[idx:], tokenizer) # idx부터 끝까지


    return train_dataset, valid_dataset, index_to_label

In [ ]:
def main(config):
    # pretrain된 tokenizer 불러오기
    tokenizer = BertTokenizerFast.from_pretrained(config.pretrained_model_name)

    # datasets과 index_to_label 받기
    train_dataset, valid_dataset, index_to_label = get_datasets(
        config.train_fn,
         tokenizer,
        valid_ratio=config.valid_ratio,
    )

    print(
        '## train ## =', len(train_dataset),
        '## valid ## =', len(valid_dataset),
    )

    total_batch_size = config.batch_size_per_device * torch.cuda.device_count() # 전체 batch size
    n_total_iterations = int(len(train_dataset) / total_batch_size * config.n_epochs) # 전체 iteration 수
    n_warmup_steps = int(n_total_iterations * config.warmup_ratio) # warmup step 수

    print(
        '## total_iters ## =', n_total_iterations,
        '## warmup_iters ## =', n_warmup_steps,
    )

    # pretrained_model_name에 따라서 model loader를 다르게 설정
    model_loader = BertForSequenceClassification
    # pretrained model 불러오기
    model = model_loader.from_pretrained(
        config.pretrained_model_name, # pretrained model 이름
        num_labels=len(index_to_label) # output label 개수
    )

    training_args = TrainingArguments(
        output_dir='./.checkpoints', # checkpoint 저장 경로
        num_train_epochs=config.n_epochs, # epoch 수
        per_device_train_batch_size=config.batch_size_per_device, # device 당 train batch size
        per_device_eval_batch_size=config.batch_size_per_device, # device 당 eval batch size
        warmup_steps=n_warmup_steps, # warmup step 수
        weight_decay=0.01, # weight decay
        fp16=True, # AMP 사용 여부
        evaluation_strategy='epoch', # epoch 단위로 eval
        save_strategy = 'epoch', # epoch 단위로 저장
        logging_steps=n_total_iterations // 100, # logging step 수
        load_best_model_at_end=True, # best model 불러오기 여부
    )

    def compute_metrics(pred):
        # prediction과 label을 받아서 accuracy 계산
        labels = pred.label_ids # np.ndarray
        # pred.predictions : label 예측값 (np.ndarray)
        preds = pred.predictions.argmax(-1) # 가장 높은 확률을 가진 label 예측값의 index

        return {
            'accuracy': accuracy_score(labels, preds)
        }

    # NLP는 미니 배치 안에 각 문장들의 길이가 다르기 때문에 padding을 해줘야 함
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 동적 padding을 위한 data collator

    # trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator, # 배치마다 패딩
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        compute_metrics=compute_metrics, # metric 계산
    )
    # train 시작
    trainer.train()

    torch.save({
        'bert': trainer.model.state_dict(), # best model의 weight 저장
        'config': config,
        'vocab': None,
        'classes': index_to_label,
        'tokenizer': tokenizer,
    }, config.model_fn)


In [ ]:
if __name__ == '__main__':
    config = define_argparser()
    main(config)